# PREDICT DATA

In [1]:
#r "nuget:Microsoft.ML,1.5.2"
#r "nuget:Microsoft.ML.LightGBM,1.5.2"
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.LightGbm;

MLContext mlContext = new MLContext(seed: 1);

Installed package Microsoft.ML version 1.5.2

Installed package Microsoft.ML.LightGBM version 1.5.2

In [1]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.csx" 

const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

var shuffledData = mlContext.Data.ShuffleRows(data, seed: 1);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.3);
var trainingData = split.TrainSet;
var testingData = split.TestSet; 

In [1]:
var featureColumns = new[] { "Temperature", "Luminosity", "Infrared", "Distance"};

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    //.Append(mlContext.Transforms.NormalizeMinMax("Features")) // https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.trainers.lightgbm.lightgbmmulticlasstrainer?view=ml-dotnet
    .Append(mlContext.MulticlassClassification.Trainers.LightGbm("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

var model = trainingPipeline.Fit(trainingData);

### Evaluate the model 

In [1]:
using Microsoft.AspNetCore.Html;

public static class Formatters
{
    public static string[] Categories { get; set; }

    public static void Register<T>(object[] parameters = null)
    {
        switch (typeof(T))
        {
            case Type cfType when cfType == typeof(ConfusionMatrix):
                RegisterConfusionMatrix(parameters);
                break;

            case Type cfdwType when cfdwType == typeof(ConfusionMatrixDisplayView):
                RegisterConfusionMatrixDisplayView();
                break;

            case Type mcmType when mcmType == typeof(MulticlassClassificationMetrics):
                RegisterMulticlassClassificationMetrics(parameters);
                break;

            case Type mcmdwType when mcmdwType == typeof(MulticlassClassificationMetricsDisplayView):
                RegisterMulticlassClassificationMetricsDisplayView();
                break;

            case Type lmcmType when lmcmType == typeof(List<TrainCatalogBase.CrossValidationResult<MulticlassClassificationMetrics>>):
                RegisterListMulticlassClassificationMetrics();
                break;

            default:
                break;
        }
    }

    private static void RegisterListMulticlassClassificationMetrics()
    {
        Formatter.Register<List<TrainCatalogBase.CrossValidationResult<MulticlassClassificationMetrics>>>((crossValidationResults, writer) =>
        {
            var metricsInMultipleFolds = crossValidationResults.Select(r => r.Metrics);
            var microAccuracyValues = ExtractMetrics(metricsInMultipleFolds.Select(m => m.MicroAccuracy));
            var macroAccuracyValues = ExtractMetrics(metricsInMultipleFolds.Select(m => m.MacroAccuracy));
            var logLossValues = ExtractMetrics(metricsInMultipleFolds.Select(m => m.LogLoss));
            var logLossReductionValues = ExtractMetrics(metricsInMultipleFolds.Select(m => m.LogLossReduction));

            var headers = new List<IHtmlContent>
        {
            th(b("CROSS-VALIDATION: multi-class classification")),
            th(b("Average")),
            th(b("Standard deviation")),
            th(b("Confidence interval (95%)"))
        };

            var rows = new List<List<IHtmlContent>>();

            var cells = new List<IHtmlContent>
        {
            td(b("MacroAccuracy")),
            td($"{macroAccuracyValues.average:0.000}"),
            td($"{macroAccuracyValues.stdDev:0.000}"),
            td($"{macroAccuracyValues.confInt:0.000}")
        };
            rows.Add(cells);

            cells = new List<IHtmlContent>
        {
            td(b("MicroAccuracy")),
            td($"{microAccuracyValues.average:0.000}"),
            td($"{microAccuracyValues.stdDev:0.000}"),
            td($"{microAccuracyValues.confInt:0.000}")
        };
            rows.Add(cells);

            cells = new List<IHtmlContent>
        {
            td(b("LogLoss")),
            td($"{logLossValues.average:0.000}"),
            td($"{logLossValues.stdDev:0.000}"),
            td($"{logLossValues.confInt:0.000}")
        };
            rows.Add(cells);

            cells = new List<IHtmlContent>
        {
            td(b("LogLossReduction")),
            td($"{logLossReductionValues.average:0.000}"),
            td($"{logLossReductionValues.stdDev:0.000}"),
            td($"{logLossReductionValues.confInt:0.000}")
        };
            rows.Add(cells);

            var t = table(
                thead(
                    headers),
                tbody(
                    rows.Select(
                        r => tr(r))));
            writer.Write(t);
        }, "text/html");

        Console.WriteLine("List<TrainCatalogBase.CrossValidationResult<MulticlassClassificationMetrics>> formatter loaded.");
    }

    private static void RegisterMulticlassClassificationMetrics(object[] parameters)
    {
        Formatter.Register<MulticlassClassificationMetrics>((m, writer) =>
        {
            if (parameters?.Length == m.PerClassLogLoss.Count)
            {
                string[] categories = new string[m.PerClassLogLoss.Count];

                for (int i = 0; i < parameters.Count(); i++)
                {
                    categories[i] = parameters[i].ToString();
                }

                var oneMessage = "the closer to 1, the better";
                var zeroMessage = "the closer to 0, the better";

                var headers = new List<IHtmlContent>
        {
            th(b("EVALUATION: multi-class classification")),
            th(b("Class")),
            th(b("Value")),
            th(b("Note"))
        };

                var rows = new List<List<IHtmlContent>>();

                var cells = new List<IHtmlContent>
        {
            td(b("MacroAccuracy")),
            td(""),
            td($"{m.MacroAccuracy:0.000}"),
            td(oneMessage)
        };
                rows.Add(cells);

                cells = new List<IHtmlContent>
        {
            td(b("MicroAccuracy")),
            td(""),
            td($"{m.MicroAccuracy:0.000}"),
            td(oneMessage)
        };
                rows.Add(cells);

                cells = new List<IHtmlContent>
        {
            td(b("LogLoss")),
            td(""),
            td($"{m.LogLoss:0.000}"),
            td(zeroMessage)
        };
                rows.Add(cells);

                cells = new List<IHtmlContent>
        {
            td[rowspan: $"{m.PerClassLogLoss.Count + 1}"](b("LogLoss per Class"))
        };
                rows.Add(cells);

                for (int i = 0; i < m.PerClassLogLoss.Count; i++)
                {
                    cells = new List<IHtmlContent>
            {
                td($"{categories[i]}"),
                td($"{m.PerClassLogLoss[i]:0.000}"),
                td(zeroMessage)
            };
                    rows.Add(cells);
                }

                var t = table(
                    thead(
                        headers),
                    tbody(
                        rows.Select(
                            r => tr(r))));
                writer.Write(t);
            }
            else 
            {
                writer.Write($"The number of classes by Confusion Matrix ({m.PerClassLogLoss.Count}) does not match the number of categories argument ({parameters?.Length})");
            }
        }, "text/html");

        Console.WriteLine("MulticlassClassificationMetrics formatter loaded.");
    }

    private static void RegisterMulticlassClassificationMetricsDisplayView()
    {
        Formatter.Register<MulticlassClassificationMetricsDisplayView>((m, writer) =>
        {
            if (m.Categories?.Length == m.Metrics.PerClassLogLoss.Count)
            {
                string[] categories = new string[m.Metrics.PerClassLogLoss.Count];

                for (int i = 0; i < m.Categories.Count(); i++)
                {
                    categories[i] = m.Categories[i].ToString();
                }

                var oneMessage = "the closer to 1, the better";
                var zeroMessage = "the closer to 0, the better";

                var headers = new List<IHtmlContent>
        {
            th(b("EVALUATION: multi-class classification")),
            th(b("Class")),
            th(b("Value")),
            th(b("Note"))
        };

                var rows = new List<List<IHtmlContent>>();

                var cells = new List<IHtmlContent>
        {
            td(b("MacroAccuracy")),
            td(""),
            td($"{m.Metrics.MacroAccuracy:0.000}"),
            td(oneMessage)
        };
                rows.Add(cells);

                cells = new List<IHtmlContent>
        {
            td(b("MicroAccuracy")),
            td(""),
            td($"{m.Metrics.MicroAccuracy:0.000}"),
            td(oneMessage)
        };
                rows.Add(cells);

                cells = new List<IHtmlContent>
        {
            td(b("LogLoss")),
            td(""),
            td($"{m.Metrics.LogLoss:0.000}"),
            td(zeroMessage)
        };
                rows.Add(cells);

                cells = new List<IHtmlContent>
        {
            td[rowspan: $"{m.Metrics.PerClassLogLoss.Count + 1}"](b("LogLoss per Class"))
        };
                rows.Add(cells);

                for (int i = 0; i < m.Metrics.PerClassLogLoss.Count; i++)
                {
                    cells = new List<IHtmlContent>
            {
                td($"{categories[i]}"),
                td($"{m.Metrics.PerClassLogLoss[i]:0.000}"),
                td(zeroMessage)
            };
                    rows.Add(cells);
                }

                var t = table(
                    thead(headers),
                    tbody(rows.Select(r => tr(r))));
                writer.Write(t);
            }
            else
            {
                writer.Write($"The number of classes by Correlation Matrix ({m.Metrics.PerClassLogLoss.Count}) does not match the number of categories argument ({m.Categories?.Length})");
            }
        }, "text/html");

        Console.WriteLine("MulticlassClassificationMetrics formatter loaded.");
    }

    private static void RegisterConfusionMatrixDisplayView()
    {
        Formatter.Register<ConfusionMatrixDisplayView>((cm, writer) =>
        {
            if (cm.Categories?.Length == cm.ConfusionMatrix.NumberOfClasses)
            {
                string[] categories = new string[cm.ConfusionMatrix.NumberOfClasses];

                for (int i = 0; i < cm.Categories.Count(); i++)
                {
                    categories[i] = cm.Categories[i].ToString();
                }

                var cssFirstColor = "background-color: lightsteelblue; ";
                var cssSecondColor = "background-color: #E3EAF3; ";
                var cssTransparent = "background-color: transparent";
                var cssBold = "font-weight: bold; ";
                var cssPadding = "padding: 8px; ";
                var cssCenterAlign = "text-align: center; ";
                var cssTable = "margin: 50px; ";
                var cssTitle = cssPadding + cssFirstColor;
                var cssHeader = cssPadding + cssBold + cssSecondColor;
                var cssCount = cssPadding;
                var cssFormula = cssPadding + cssSecondColor;

                var rows = new List<IHtmlContent>();

                // header
                var cells = new List<IHtmlContent>
        {
            td[rowspan: 2, colspan: 2, style: cssTitle + cssCenterAlign]("Confusion Matrix"),
            td[colspan: cm.ConfusionMatrix.Counts.Count, style: cssTitle + cssCenterAlign]("Predicted"),
            td[style: cssTitle]("")
        };
                rows.Add(tr[style: cssTransparent](cells));

                // features header
                cells = new List<IHtmlContent>();
                for (int j = 0; j < cm.ConfusionMatrix.Counts.Count; j++)
                {
                    cells.Add(td[style: cssHeader](categories.ToList()[j]));
                }
                rows.Add(tr[style: cssTransparent](cells));
                cells.Add(td[style: cssTitle]("Recall"));

                // values
                for (int i = 0; i < cm.ConfusionMatrix.NumberOfClasses; i++)
                {
                    cells = new List<IHtmlContent>();
                    if (i == 0)
                    {
                        cells.Add(td[rowspan: cm.ConfusionMatrix.Counts.Count, style: cssTitle]("Truth"));
                    }
                    cells.Add(td[style: cssHeader](categories.ToList()[i]));
                    for (int j = 0; j < cm.ConfusionMatrix.NumberOfClasses; j++)
                    {
                        cells.Add(td[style: cssCount](cm.ConfusionMatrix.Counts[i][j]));
                    }
                    cells.Add(td[style: cssFormula](Math.Round(cm.ConfusionMatrix.PerClassRecall[i], 4)));
                    rows.Add(tr[style: cssTransparent](cells));
                }

                //footer
                cells = new List<IHtmlContent>
        {
            td[colspan: 2, style: cssTitle]("Precision")
        };
                for (int j = 0; j < cm.ConfusionMatrix.Counts.Count; j++)
                {
                    cells.Add(td[style: cssFormula](Math.Round(cm.ConfusionMatrix.PerClassPrecision[j], 4)));
                }
                cells.Add(td[style: cssFormula]("total = " + cm.ConfusionMatrix.Counts.Sum(x => x.Sum())));
                rows.Add(tr[style: cssTransparent](cells));

                writer.Write(table[style: cssTable](tbody(rows)));
            }
            else
            {
                writer.Write($"The number of classes in the Confusion Matrix ({cm.ConfusionMatrix.NumberOfClasses}) does not match the number of categories argument ({cm.Categories?.Length})");
            }

        }, "text/html");

        Console.WriteLine("ConfusionMatrix formatter loaded.");
    }

    private static void RegisterConfusionMatrix(object[] parameters)
    {
        Formatter.Register<ConfusionMatrix>((cm, writer) =>
        {
            if (parameters?.Length == cm.NumberOfClasses)
            {
                string[] categories = new string[cm.NumberOfClasses];

                for (int i = 0; i < parameters.Count(); i++)
                {
                    categories[i] = parameters[i].ToString();
                }

                var cssFirstColor = "background-color: lightsteelblue; ";
                var cssSecondColor = "background-color: #E3EAF3; ";
                var cssTransparent = "background-color: transparent";
                var cssBold = "font-weight: bold; ";
                var cssPadding = "padding: 8px; ";
                var cssCenterAlign = "text-align: center; ";
                var cssTable = "margin: 50px; ";
                var cssTitle = cssPadding + cssFirstColor;
                var cssHeader = cssPadding + cssBold + cssSecondColor;
                var cssCount = cssPadding;
                var cssFormula = cssPadding + cssSecondColor;

                var rows = new List<IHtmlContent>();

                // header
                var cells = new List<IHtmlContent>
        {
            td[rowspan: 2, colspan: 2, style: cssTitle + cssCenterAlign]("Confusion Matrix"),
            td[colspan: cm.Counts.Count, style: cssTitle + cssCenterAlign]("Predicted"),
            td[style: cssTitle]("")
        };
                rows.Add(tr[style: cssTransparent](cells));

                // features header
                cells = new List<IHtmlContent>();
                for (int j = 0; j < cm.Counts.Count; j++)
                {
                    cells.Add(td[style: cssHeader](categories.ToList()[j]));
                }
                rows.Add(tr[style: cssTransparent](cells));
                cells.Add(td[style: cssTitle]("Recall"));

                // values
                for (int i = 0; i < cm.NumberOfClasses; i++)
                {
                    cells = new List<IHtmlContent>();
                    if (i == 0)
                    {
                        cells.Add(td[rowspan: cm.Counts.Count, style: cssTitle]("Truth"));
                    }
                    cells.Add(td[style: cssHeader](categories.ToList()[i]));
                    for (int j = 0; j < cm.NumberOfClasses; j++)
                    {
                        cells.Add(td[style: cssCount](cm.Counts[i][j]));
                    }
                    cells.Add(td[style: cssFormula](Math.Round(cm.PerClassRecall[i], 4)));
                    rows.Add(tr[style: cssTransparent](cells));
                }

                //footer
                cells = new List<IHtmlContent>
        {
            td[colspan: 2, style: cssTitle]("Precision")
        };
                for (int j = 0; j < cm.Counts.Count; j++)
                {
                    cells.Add(td[style: cssFormula](Math.Round(cm.PerClassPrecision[j], 4)));
                }
                cells.Add(td[style: cssFormula]("total = " + cm.Counts.Sum(x => x.Sum())));
                rows.Add(tr[style: cssTransparent](cells));

                writer.Write(table[style: cssTable](tbody(rows)));
            }
            else
            {
                writer.Write($"The number of classes in the Confusion Matrix ({cm.NumberOfClasses}) does not match the number of categories argument ({parameters?.Length})");
            }

        }, "text/html");

        Console.WriteLine("ConfusionMatrix formatter loaded.");
    }

    private static (double average, double stdDev, double confInt) ExtractMetrics(IEnumerable<double> accuracyValues)
    {
        var average = accuracyValues.Average();
        var stdDev = CalculateStandardDeviation(accuracyValues);
        var confInt = CalculateConfidenceInterval95(accuracyValues);

        return (average, stdDev, confInt);
    }

    private static double CalculateStandardDeviation(IEnumerable<double> values)
    {
        double average = values.Average();
        double sumOfSquaresOfDifferences = values.Select(val => (val - average) * (val - average)).Sum();
        double standardDeviation = Math.Sqrt(sumOfSquaresOfDifferences / (values.Count() - 1));

        return standardDeviation;
    }

    private static double CalculateConfidenceInterval95(IEnumerable<double> values)
    {
        double confidenceInterval95 = 1.96 * CalculateStandardDeviation(values) / Math.Sqrt(values.Count() - 1);

        return confidenceInterval95;
    }
}

public class ConfusionMatrixDisplayView
{
    public ConfusionMatrix ConfusionMatrix { get; set; }
    public string[] Categories { get; set; }
}

public class MulticlassClassificationMetricsDisplayView
{
    public MulticlassClassificationMetrics Metrics { get; set; }
    public string[] Categories { get; set; }
}

In [1]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

Formatters.Categories = new string[] { "FlashLight", "Infrared", "DayLight", "Lighter" };
Formatters.Register<ConfusionMatrix>(Formatters.Categories);
Formatters.Register<MulticlassClassificationMetrics>(Formatters.Categories);

var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

display(metrics);

display(metrics.ConfusionMatrix);

display(p[@style:"font-size: 40px;"]($"MicroAccuracy: {metrics.MicroAccuracy:P2}, MacroAccuracy: {metrics.MacroAccuracy:P2}"));

Installed package Microsoft.Data.Analysis version 0.4.0

ConfusionMatrix formatter loaded.


MulticlassClassificationMetrics formatter loaded.


EVALUATION: multi-class classification Class Value Note MacroAccuracy 0.918 the closer to 1, the better MicroAccuracy 0.923 the closer to 1, the better LogLoss 0.323 the closer to 0, the better LogLoss per Class FlashLight 0.353 the closer to 0, the better Infrared 0.620 the closer to 0, the better DayLight 0.194 the closer to 0, the better Lighter 0.213 the closer to 0, the better

Confusion Matrix Predicted FlashLight Infrared DayLight Lighter Recall Truth FlashLight 70 1 2 2 0.9333 Infrared 0 52 6 2 0.8667 DayLight 0 1 103 5 0.945 Lighter 1 0 3 51 0.9273 Precision 0.9859 0.963 0.9035 0.85 total = 299

MicroAccuracy: 92.31%, MacroAccuracy: 91.81%

## Predict data

In [1]:
#r "nuget:MathNet.Numerics"
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Helpers.csx"

var sampleData = new ModelInput
{
    Temperature = 32F,
    Luminosity = 22F,
    Infrared = 0F,
    Distance = 20F,
    PIR = 1F,
    Humidity = 13F,
    CreatedAt = "01/03/2020 10:22:08"
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[@style:"font-size: 40px"]($"{predicted.PredictedLabel}"));

display(Helpers.GetPredictionPerClass(predicted, predictor.OutputSchema));

FlashLight

0.9320,FlashLight
0.0625,DayLight
0.0051,Infrared
0.0003,Lighter
